<a href="https://colab.research.google.com/github/ambgeo/geoquantificacao/blob/main/01_Fundamentos_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛰️ Fundamentos do Google Earth Engine (GEE) — Notebook Simples

Este notebook cobre, de forma direta, os fundamentos solicitados:
* 1) **O que é o GEE**
* 2) **Autenticação**
* 3) **Abrir um mapa no geemap**
* 4) **Definir a ROI a partir de um asset em `users/...`**
* 5) **Abrir o dado do NASADEM**
* 6) **Extrair média, mínimo e máximo para a ROI**
* 7) **Plotar um gráfico usando `ee_to_df`**
* 8) **Exportar a cena**

## 1) O que é o GEE
O **Google Earth Engine (GEE)** é uma plataforma de computação em nuvem para análise de dados geoespaciais em escala.
Ela reúne um grande repositório de dados (imagens de satélite, modelos, vetores) e oferece uma API para processar tudo **no servidor**, sem precisar baixar dados localmente.

## 2) Autenticação
Execute a célula abaixo. Se for a primeira vez na sessão, será solicitado login.

> Observação: o `project` abaixo está definido como `ee-scriptsambgeo` conforme solicitado. Ajuste se necessário.

In [ ]:
import ee

# 1) Autenticar (rode uma vez por sessão):
try:
    ee.Authenticate()  # Pode abrir uma janela para login; copie e cole o token.
except Exception as e:
    print('Se já autenticou anteriormente, você pode ignorar este aviso:', e)

# 2) Inicializar a sessão do Earth Engine
ee.Initialize(project='ee-scriptsambgeo')
print('GEE inicializado com sucesso!')

## 3) Abrir um mapa no geemap
Crie um mapa interativo. Se preferir, ajuste o centro/zoom.

In [ ]:
# Se necessário: !pip install -q geemap folium ipyleaflet
import geemap

Map = geemap.Map(center=[-15, -55], zoom=4)
Map.addLayerControl()
Map

## 4) Definir a ROI desenhando no mapa

- Use a ferramenta **Draw** do `geemap` (ícone de lápis no canto superior esquerdo do mapa) para desenhar um **polígono** delimitando sua área de interesse (ROI).
- Após desenhar, execute a célula abaixo para capturar a geometria e usá-la nas análises.
- A ROI será definida diretamente a partir das coordenadas desenhadas no mapa.

> Dica: você pode apagar ou redesenhar a ROI no mapa a qualquer momento antes de executar a célula.


In [ ]:
roi = Map.user_roi
Map.centerObject(roi, 9)
Map.addLayer(roi, {"color": "red"}, "ROI")
Map

## 5) Abrir dado do NASADEM
Usaremos o dataset **`NASA/NASADEM_HGT/001`** e a banda `elevation` (aprox. 30 m).

In [ ]:
nasadem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
nasadem_roi = nasadem.clip(roi) if roi is not None else nasadem

viz = {"min": 0, "max": 3000}
Map.addLayer(nasadem_roi, viz, 'NASADEM — Elevação')
Map

## 6) Estatísticas (média, mínimo e máximo) na ROI
Calculamos estatísticas simples de elevação dentro da ROI.

In [ ]:
if roi is None:
    raise ValueError('Defina a ROI antes de prosseguir (asset users/... ou desenho no mapa).')

stats = nasadem.reduceRegion(
    reducer=ee.Reducer.mean().combine(ee.Reducer.minMax(), sharedInputs=True),
    geometry=roi,
    scale=30,
    maxPixels=1e12
).getInfo()
print('Estatísticas de Elevação (m):', stats)

## 7) Gráfico com `ee_to_df`
Amostramos pontos dentro da ROI, convertemos para **DataFrame** com `geemap.ee_to_df()` e plotamos um histograma simples de elevação.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Amostra de pixels (reduza numPixels para ROIs muito grandes)
samples_fc = nasadem.sample(region=roi, scale=30, numPixels=2000, geometries=True, seed=42)
df = geemap.ee_to_df(samples_fc)

df.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,4))
plt.hist(df['elevation'].dropna(), bins=30)
plt.title('Distribuição de Elevação (NASADEM)')
plt.xlabel('Elevação (m)')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()

## 8) Exportar a cena
Você pode exportar a imagem recortada pela ROI como **GeoTIFF**.

### Opção A) Exportar para Google Drive (servidor do GEE)
Use `ee.batch.Export.image.toDrive` e acompanhe o status na aba **Tasks** do Code Editor.

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=nasadem_roi,
    description='NASADEM_elevacao_ROI',
    fileNamePrefix='NASADEM_elevacao_ROI',
    region=roi.getInfo()['coordinates'],
    folder='Geoquanti',
    scale=30,
    maxPixels=1e13
)
task.start()
print('Exportação iniciada para o Google Drive. Verifique a aba **Tasks** no EE Code Editor.')

In [ ]:
##status
task.status()

### Opção B) Exportar localmente com `geemap`
Baixa o GeoTIFF diretamente para a máquina onde o notebook está rodando.

In [ ]:
import geemap
out_tif = 'nasadem_elevacao_roi.tif'
geemap.ee_export_image(
    nasadem_roi,
    filename=out_tif,
    scale=30,
    region=roi,
    file_per_band=False
)
print('Arquivo salvo em:', out_tif)

## 9) Plot rasterio

In [ ]:
!pip -q install rasterio spectral

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

with rio.open('/content/nasadem_elevacao_roi.tif') as src:
    elev = src.read(1, masked=True)
    left, bottom, right, top = src.bounds  # coordenadas reais do raster

# stretch robusto para contraste
vmin, vmax = np.nanpercentile(elev.compressed(), (2, 98))

plt.figure(figsize=(6,5))
plt.imshow(elev, cmap='terrain', vmin=vmin, vmax=vmax,
           extent=[left, right, bottom, top], origin='upper')
plt.colorbar(label='Elevação (m)')
plt.xlabel(f'Longitude ({src.crs.to_string()})')
plt.ylabel('Latitude')
plt.title('NASADEM — Elevação (m)')
plt.show()
